## The CrossTree

Votiong Schema with mulitple binary classification tree. The network implement votion schema based on three different tree:

* Cultural **Agnostic-Rappresentative** tree
* Cultural **Agnostic-Exclusive** tree
* Cultural **Exclusive-Rappresentative** tree

majority voting detterminate final class.

### Training Phase

The training process is quite standard and straight-forward: given the n G_features we want to directly predict the associated class.

### Employment Phase

The training model will be inserted in a wider model called X and utilized as a function for the computation of the G_Factor

## Dataset

Load the dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
def onehot_encode(
    df_train: pd.DataFrame,
    df_test: pd.DataFrame,
    cat_cols: list[str]|None = None,
    num_cols: list[str]|None = None,
    sparse: bool = False
) -> tuple[pd.DataFrame, pd.DataFrame, OneHotEncoder]:
    """
    Applica One-Hot Encoding a df_train e df_test garantendo lo stesso
    insieme di colonne, anche se in train mancano categorie presenti in test.

    Parameters
    ----------
    df_train : pd.DataFrame
        DataFrame di training.
    df_test : pd.DataFrame
        DataFrame di test.
    cat_cols : list[str], optional
        Lista delle colonne categoriche da codificare. 
        Se None, vengono prese tutte le colonne di tipo 'object'.
    num_cols : list[str], optional
        Lista delle colonne numeriche (o non categoriche) da preservare. 
        Se None, vengono calcolate come tutte le colonne non in cat_cols.
    handle_unknown : str, default="ignore"
        Comportamento su valori sconosciuti in test (tipicamente "ignore").
    sparse : bool, default=False
        Se True restituisce matrice sparsa, altrimenti densa.

    Returns
    -------
    df_train_enc : pd.DataFrame
        DataFrame di train con One-Hot Encoding + num_cols originali.
    df_test_enc : pd.DataFrame
        DataFrame di test con One-Hot Encoding + num_cols originali.
    encoder : OneHotEncoder
        L’oggetto già fit su train+test, utile per futuri trasform.
    """
    # 1) Individuo le colonne categoriche e numeriche (se non date)
    if cat_cols is None:
        cat_cols = df_train.select_dtypes(include="object").columns.tolist()
    if num_cols is None:
        num_cols = [c for c in df_train.columns if c not in cat_cols]

    # 2) Fitto l’encoder su tutti i dati categorici (train + test)
    all_cats = pd.concat([df_train[cat_cols], df_test[cat_cols]], 
                         axis=0, ignore_index=True)
    encoder = OneHotEncoder(
        sparse_output=sparse
    ).fit(all_cats)

    # 3) Trasformo separatamente train e test
    X_train_ohe = encoder.transform(df_train[cat_cols])
    X_test_ohe  = encoder.transform(df_test[cat_cols])

    # 4) Nominiamo le nuove colonne
    ohe_cols = encoder.get_feature_names_out(cat_cols).tolist()

    # 5) Ricompongo i DataFrame finali
    df_train_enc = pd.DataFrame(
        np.hstack([X_train_ohe.toarray() if sparse else X_train_ohe,
                   df_train[num_cols].values]), # type: ignore
        columns=ohe_cols + num_cols,
        index=df_train.index
    )
    df_test_enc = pd.DataFrame(
        np.hstack([X_test_ohe.toarray() if sparse else X_test_ohe,
                   df_test[num_cols].values]),
        columns=ohe_cols + num_cols,
        index=df_test.index
    )

    return df_train_enc[ohe_cols], df_test_enc[ohe_cols], encoder


In [ ]:
import pandas as pd
import numpy as np
from CU_Dataset_Factory import Hf_Loader, CU_Dataset_Factory

In [ ]:
#print('Cultural Dataset argumentation start')
#factory = CU_Dataset_Factory('.')
#train_l = Hf_Loader("sapienzanlp/nlp2025_hw1_cultural_dataset", 'train')
#validation_l = Hf_Loader("sapienzanlp/nlp2025_hw1_cultural_dataset", 'validation')

#factory.produce(train_l, 'train.tsv', ['category', 'subcategory','type','languages','reference','num_langs'], 'label', 10, False)
#factory.produce(validation_l, 'validation.tsv', ['category', 'subcategory','type','languages','reference','num_langs'], 'label', 10, False)
#print('End process')

In [ ]:
train = pd.read_csv('train.tsv', sep='\t')
validation = pd.read_csv('validation.tsv', sep='\t')

In [ ]:
train.head(5)

In [ ]:
validation.head(5)

In [ ]:
y_train = train[['label']]
y_validation = validation[['label']]

id_train = train[['wiki_name']]
id_validation = validation[['wiki_name']]

fe_train = train[['languages', 'num_langs', 'reference']]
fe_validation = validation[['languages', 'num_langs', 'reference']]

fe_str_train = train[['category', 'subcategory', 'type']]
fe_str_validation = validation[['category', 'subcategory', 'type']]

In [ ]:
train_cat, validation_cat, _ =  onehot_encode(fe_str_train, fe_str_validation, ['category'] )
train_scat, validation_scat, _ = onehot_encode(fe_str_train, fe_str_validation, ['subcategory'] )
train_t, validation_t, _ = onehot_encode(fe_str_train, fe_str_validation, ['type'] )


In [ ]:
print(validation_cat.shape)
print(validation_scat.shape)
print(validation_t.shape)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
#pca = PCA(n_components=10)
#validation_cat = pd.DataFrame(pca.fit_transform(validation_cat))
#train_cat = pd.DataFrame(pca.fit_transform(train_cat))
#pca = PCA(n_components=50)
#validation_scat = pd.DataFrame(pca.fit_transform(validation_scat))
#train_scat = pd.DataFrame(pca.fit_transform(train_scat))


In [ ]:
print(validation_cat.shape)
print(validation_scat.shape)
print(validation_t.shape)

In [ ]:
train = pd.concat([fe_train, train_cat, train_scat, train_t, y_train], axis=1)
validation = pd.concat([fe_validation, validation_cat, validation_scat, validation_t, y_validation], axis=1) 

print(train.shape)

In [ ]:
train.head(3)

In [ ]:
validation.head(3)

## Network

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

### Agnostic-Rappresentative classifier

In [ ]:
# prepare  il dataset in modo da prendere solo due classi ed elimina gli identificativi degli elementi
d = train.query("label == 0 or label == 1")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [ ]:
ar_tree = MLPClassifier().fit(x, y)

In [ ]:
d = validation.query("label == 0 or label == 1")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [ ]:
y_pred = ar_tree.predict(x)
print(classification_report(y, y_pred))

## Agnostic-Exclusive Classifier

In [ ]:
# prepare  il dataset in modo da prendere solo due classi ed elimina gli identificativi degli elementi
d = train.query("label == 0 or label == 2")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [ ]:
ae_tree = MLPClassifier().fit(x, y)

In [ ]:
d = validation.query("label == 0 or label == 2")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [ ]:
y_pred = ae_tree.predict(x)
print(classification_report(y, y_pred))

## Rappresentative-Exclusive

In [ ]:
d = train.query("label == 1 or label == 2")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
re_tree = RandomForestClassifier().fit(x, y)

In [ ]:
d = validation.query("label == 1 or label == 2")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [ ]:
y_pred = re_tree.predict(x)
print(classification_report(y, y_pred))

# Voting Schema


In [ ]:
model = VotingClassifier([ ('agnostic_rapresentative',ar_tree), ('agnostic_exsclusive',ae_tree), ('rappresentative_exsclusive',re_tree)])

In [ ]:
from scipy.stats import mode
from sklearn.base import BaseEstimator
class Model(BaseEstimator):
    def __init__(self, t1:MLPClassifier,t2:MLPClassifier,t3:MLPClassifier) -> None:
        
        self.t1 = t1 
        self.t2 = t2 
        self.t3 = t3
    
    def fit(self, X, y):
        
        
        return self
    def predict(self, X:pd.DataFrame|np.ndarray) -> np.ndarray:
        v1 = self.t1.predict(X)
        v2 = self.t2.predict(X)
        v3 = self.t3.predict(X)
        
        # 2) impila in un array (3, n_samples)
        votes = np.vstack([v1, v2, v3])

        # 3) funzione che prende la classe più frequente in un vettore di voti
        def majority_vote(col):
            counts = np.bincount(col)
            return np.argmax(counts)

        majority, _ = mode(votes, axis=0)
        return majority.ravel()

In [ ]:
from scipy.stats import mode
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.ensemble import IsolationForest
# Cultural-classification Network
class CABNet(BaseEstimator, ClassifierMixin):
    def __init__(self, ar_estimator, ae_estimator, re_estimator) -> None:
        
        self.ar = ar_estimator 
        self.ae = ae_estimator 
        self.re = re_estimator
        
    
    def __fit(self, e, X, y) -> BaseEstimator:
        return e.fit(X, y)


    def fit(self, X, y=None):
       
        d = X.query("label == 0 or label == 1 ")
        y = d['label'].astype(int).to_numpy()
        x = d.drop(['label'], axis=1).astype(float).to_numpy()
        self.ar = self.__fit(self.ar, x, y)
        d = X.query("label == 0 or label == 2")
        y = d['label'].astype(int).to_numpy()
        x = d.drop(['label'], axis=1).astype(float).to_numpy()
        self.ae = self.__fit(self.ae, x, y)
        d = X.query("label == 1 or label == 2")
        y = d['label'].astype(int).to_numpy()
        x = d.drop(['label'], axis=1).astype(float).to_numpy()
        self.re = self.__fit(self.re, x, y)
        return self
        
    def predict(self, X):
        v1 = self.ar.predict(X)
        v2 = self.ae.predict(X)
        v3 = self.re.predict(X)
       
  
        votes = np.vstack([v1, v2, v3])
        majority, _ = mode(votes, axis=0)
        return majority.ravel()

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, IsolationForest
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier



In [ ]:
model =  CABNet(RandomForestClassifier(n_estimators=100), RandomForestClassifier(n_estimators=500), RandomForestClassifier(n_estimators=500))

In [ ]:
model = model.fit(train)

In [ ]:
d = validation
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [ ]:
y_pred = model.predict(x)
print(classification_report(y, y_pred))